In [5]:
from twikit import Client
import pandas as pd
from config import USERNAME, EMAIL, PASSWORD
# Initialize client
client = Client('en-UK')

client.login(
    auth_info_1=USERNAME ,
    auth_info_2=EMAIL,
    password=PASSWORD
)

# Initialize an empty list to store tweet data
tweets_data = []

# Search for tweets
tweets = client.search_tweet('Trump OR Biden', 'Top', count = 100)

# Iterate over the tweets and store relevant data in the list
for tweet in tweets:
    tweets_data.append([tweet.user.name, tweet.created_at, tweet.text, tweet.user.location, tweet.favorite_count, tweet.retweet_count])

# Convert the list to a pandas DataFrame
df = pd.DataFrame(tweets_data, columns=['User', 'Date', 'Tweet', 'Location', 'Likes', 'Retweets'])

# Save the DataFrame to a CSV file
df.to_csv('tweets.csv', index=False)

print("Data collection complete. Tweets saved to climate_change_tweets.csv")

Data collection complete. Tweets saved to climate_change_tweets.csv


In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px


In [ ]:
# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1("Twitter Dashboard"),
    html.H2("In this dashboard you can view and analyse tweets about whatever topic you want."),
    html.H2("What topic would you like to search for? Split each topic by a comma."),
    dcc.Input(id='search-input', type='text', style={'width': '50%'}),
    html.H2("How many tweets would you like to collect?"),
    dcc.Input(id='count-input', type='number'),
    dcc.Dropdpwn(
        id='search-option',
        options=[
            {'label': 'Top', 'value': 'Top'},
            {'label': 'Latest', 'value': 'Latest'},
            {'label': 'Media', 'value': 'Media'}
        ],
    ),
    html.Button('Collect data', id='collect-button', n_clicks=0),
    html.Div(id='sentiment-output'),
    

    html.H2("Top 3 Most Liked Tweets"),
    html.Div(id='top-liked-tweets'),
    html.H2("Top 3 Most Retweeted Tweets"),
    html.Div(id='top-retweeted-tweets'),
    dcc.Graph(id='sentiment-bar-chart'),
    html.H2("Input Text for Sentiment Analysis"),
    dcc.Input(id='input-text', type='text', style={'width': '50%'}),
    html.Button('Analyze', id='analyze-button', n_clicks=0),
    html.Div(id='sentiment-output')
])

# Callback to update the tweet display based on selected sentiment
@app.callback(
    [Output('top-tweets-title', 'children'),
     Output('tweets-display', 'children')],
    [Input('sentiment-dropdown', 'value')]
)


# Callback to update the most liked and retweeted tweets
@app.callback(
    [Output('top-liked-tweets', 'children'),
     Output('top-retweeted-tweets', 'children')],
    [Input('sentiment-dropdown', 'value')]
)
def update_top_tweets(selected_sentiment):

    top_liked_tweets = cleaned_df.nlargest(3, 'Likes')
    top_retweeted_tweets = cleaned_df.nlargest(3, 'Retweets')

    liked_list = html.Ul([html.Li(f"{tweet['User']} (Likes: {tweet['Likes']}): {tweet['Tweet']}") for _, tweet in top_liked_tweets.iterrows()])
    retweeted_list = html.Ul([html.Li(f"{tweet['User']} (Retweets: {tweet['Retweets']}): {tweet['Tweet']}") for _, tweet in top_retweeted_tweets.iterrows()])

    return liked_list, retweeted_list

# Callback to update the sentiment bar chart
@app.callback(
    Output('sentiment-bar-chart', 'figure'),
    [Input('sentiment-dropdown', 'value')]
)
def update_bar_chart(selected_sentiment):
    sentiment_counts = df['SentimentBin'].value_counts().reset_index()
    sentiment_counts.columns = ['Sentiment', 'Count']
    fig = px.bar(sentiment_counts, x='Sentiment', y='Count', title='Sentiment Distribution')
    fig.update_layout(xaxis_title='Sentiment Category', yaxis_title='Count')
    return fig

# Callback to perform sentiment analysis on user input text
@app.callback(
    Output('sentiment-output', 'children'),
    [Input('analyze-button', 'n_clicks')],
    [dash.dependencies.State('input-text', 'value')]
)
def analyze_input_text(n_clicks, input_text):
    if n_clicks > 0 and input_text:
        sentiment_score = analyze_sentiment(input_text)
        return f"Sentiment Score: {sentiment_score}"
    return ""

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


In [14]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1("Twitter Dashboard"),
    html.H2("In this dashboard you can view and analyse tweets about whatever topic you want."),
    html.H2("What topic would you like to search for? Split each topic by a comma."),
    dcc.Input(id='search-input', type='text', style={'width': '50%'}),
    html.H2("How many tweets would you like to collect?"),
    dcc.Input(id='count-input', type='number'),
    html.H2("Select the search option:"),
    dcc.Dropdown(
        id='search-option',
        options=[
            {'label': 'Top', 'value': 'Top'},
            {'label': 'Latest', 'value': 'Latest'},
            {'label': 'Media', 'value': 'Media'}
        ], style={'width': '50%'}
    ),
    html.Button('Collect data', id='collect-button', n_clicks=0),
    html.Div(id='tweet-data-output'),
    
    html.H2("Top 3 Most Liked Tweets"),
    html.Div(id='top-liked-tweets'),
    html.H2("Top 3 Most Retweeted Tweets"),
    html.Div(id='top-retweeted-tweets'),
    dcc.Graph(id='sentiment-bar-chart'),
    html.H2("Input Text for Sentiment Analysis"),
    dcc.Input(id='input-text', type='text', style={'width': '50%'}),
    html.Button('Analyze', id='analyze-button', n_clicks=0),
    html.Div(id='sentiment-output')
])

# Create a global variable to store the DataFrame
global_df = pd.DataFrame()

@app.callback(
    Output('tweet-data-output', 'children'),
    [Input('collect-button', 'n_clicks'),
     Input('search-input', 'value'),
     Input('count-input', 'value'),
     Input('search-option', 'value')]
)
def update_tweets(n_clicks, search_input, count_input, search_option):
    global global_df
    
    if n_clicks > 0:
        if not search_input or not count_input or not search_option:
            return "Please provide search topics, number of tweets, and search option."
        
        # Reset the tweets_data list
        tweets_data = []
        
        # Split the search input by comma and join with OR
        topics = ' OR '.join(search_input.split(','))
        
        # Search for tweets (mocked for this example)
        tweets = client.search_tweet(topics, search_option, count=count_input)

        # Iterate over the tweets and store relevant data in the list
        for tweet in tweets:
            tweets_data.append([tweet.user.name, tweet.created_at, tweet.text, tweet.user.location, tweet.favorite_count, tweet.retweet_count])

        # Convert the list to a pandas DataFrame
        global_df = pd.DataFrame(tweets_data, columns=['User', 'Date', 'Tweet', 'Location', 'Likes', 'Retweets'])
        
    return ""

@app.callback(
    [Output('top-liked-tweets', 'children'),
     Output('top-retweeted-tweets', 'children')],
    Input('collect-button', 'n_clicks')
)
def update_top_tweets(n_clicks):
    global global_df
    
    if n_clicks > 0 and not global_df.empty:
        top_liked_tweets = global_df.nlargest(3, 'Likes')
        top_retweeted_tweets = global_df.nlargest(3, 'Retweets')

        liked_list = html.Ul([html.Li(f"{tweet['User']} (Likes: {tweet['Likes']}): {tweet['Tweet']}") for _, tweet in top_liked_tweets.iterrows()])
        retweeted_list = html.Ul([html.Li(f"{tweet['User']} (Retweets: {tweet['Retweets']}): {tweet['Tweet']}") for _, tweet in top_retweeted_tweets.iterrows()])
        
        return liked_list, retweeted_list

    return "", ""

if __name__ == '__main__':
    app.run_server(debug=True, port=8052)


In [15]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1("Twitter Dashboard"),
    html.H2("In this dashboard you can view and analyse tweets about whatever topic you want."),
    html.H2("What topic would you like to search for? Split each topic by a comma."),
    dcc.Input(id='search-input', type='text', style={'width': '50%'}),
    html.H2("How many tweets would you like to collect?"),
    dcc.Input(id='count-input', type='number'),
    html.H2("Select the search option:"),
    dcc.Dropdown(
        id='search-option',
        options=[
            {'label': 'Top', 'value': 'Top'},
            {'label': 'Latest', 'value': 'Latest'},
            {'label': 'Media', 'value': 'Media'}
        ], style={'width': '50%'}
    ),
    html.Button('Collect data', id='collect-button', n_clicks=0),
    html.Div(id='tweet-data-output'),
    
    html.H2("Top 3 Most Liked Tweets"),
    html.Div(id='top-liked-tweets'),
    html.H2("Top 3 Most Retweeted Tweets"),
    html.Div(id='top-retweeted-tweets'),
    dcc.Graph(id='sentiment-bar-chart'),
    html.H2("Input Text for Sentiment Analysis"),
    dcc.Input(id='input-text', type='text', style={'width': '50%'}),
    html.Button('Analyze', id='analyze-button', n_clicks=0),
    html.Div(id='sentiment-output')
])

# Create a global variable to store the DataFrame
global_df = pd.DataFrame()

@app.callback(
    [Output('tweet-data-output', 'children'),
     Output('top-liked-tweets', 'children'),
     Output('top-retweeted-tweets', 'children')],
    [Input('collect-button', 'n_clicks'),
     Input('search-input', 'value'),
     Input('count-input', 'value'),
     Input('search-option', 'value')]
)
def update_tweets(n_clicks, search_input, count_input, search_option):
    global global_df
    
    if n_clicks > 0:
        if not search_input or not count_input or not search_option:
            return ("Please provide search topics, number of tweets, and search option.", "", "")
        
        # Reset the tweets_data list
        tweets_data = []
        
        # Split the search input by comma and join with OR
        topics = ' OR '.join(search_input.split(','))
        
        # Search for tweets (mocked for this example)
        tweets = client.search_tweet(topics, search_option, count=count_input)

        # Iterate over the tweets and store relevant data in the list
        for tweet in tweets:
            tweets_data.append([tweet.user.name, tweet.created_at, tweet.text, tweet.user.location, tweet.favorite_count, tweet.retweet_count])

        # Convert the list to a pandas DataFrame
        global_df = pd.DataFrame(tweets_data, columns=['User', 'Date', 'Tweet', 'Location', 'Likes', 'Retweets'])
        
        # Find the top liked and retweeted tweets
        top_liked_tweets = global_df.nlargest(3, 'Likes')
        top_retweeted_tweets = global_df.nlargest(3, 'Retweets')

        liked_list = html.Ul([html.Li(f"{tweet['User']} (Likes: {tweet['Likes']}): {tweet['Tweet']}") for _, tweet in top_liked_tweets.iterrows()])
        retweeted_list = html.Ul([html.Li(f"{tweet['User']} (Retweets: {tweet['Retweets']}): {tweet['Tweet']}") for _, tweet in top_retweeted_tweets.iterrows()])
        
        return "", liked_list, retweeted_list

    return "", "", ""

if __name__ == '__main__':
    app.run_server(debug=True, port=8053)
